# Section I: Concepts

I'm [John Williamson](http://johnhw.com)

The purpose of this crash course is to give you enough vocabulary to be able to follow the rest of the summer school. There isn't time to cover the details of the methods I'll talk about, the historical orgins and background or much about what techinques and models you might prefer and why. 

Instead, I aim to cover just enough that you can understand the material that follows.



## What is machine learning?

### Machine learning can be summarised as making *predictions* from *data*
This is slightly distinct from statistics, which is traditionally concerned with making inferences from data -- e.g. determining if an effect is present in a scientific study. Instead, machine learning tries to estimate unknown variables given some data which might predict them. 

### Machine learning in HCI
#### Example: gesture recognition
#### Example: language modelling
#### Example: touch prediction
#### Example: sensor fusion

### Some mathematical notation



In [ ]:
# standard imports
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as axes3d
import seaborn
import sklearn.preprocessing, sklearn.cluster, sklearn.tree, sklearn.neighbors, sklearn.ensemble
import ipy_table
import sklearn.svm, sklearn.cross_validation, sklearn.grid_search, sklearn.metrics, sklearn.datasets, sklearn.decomposition, sklearn.manifold
import pandas as pd
# force plots to appear inline on this page
%matplotlib inline

## Supervised versus unsupervised learning

Supervised learning involves learning a relationship between attribute variables and target variables; in other words learning a function which maps input measurements to target values. This can be in the context of making discrete decisions (is this image a car or not?) or learning continuous relationships (

### Supervised learning
#### Classification
#### Knn example

#### Regression
### Unsupervised learning
Unsupervised learning learns the structure of data without any explicit labeling of points. The key idea is that datasets may have a simple underlying or *latent* representation which can be determined simply by looking at the data itself.

Two common unsupervised learning tasks are *clustering* and *dimensional reduction*. Clustering can be thought of as the unsupervised analogue of classification -- finding discrete classes in data. Dimensional reduction can be thought of as the analogue of regression -- finding a small set of continuous variables which "explain" a higher dimensional set. 

#### Learning-to-rank

#### Generative models versus black box models
##### Tank recognition

#### Dimensional reduction



#### Principal component analysis
One very simple method of dimensional reduction is *principal component analysis*. This is a linear method; in other words it finds rigid rotations and scalings of the data to project it onto a lower dimension.

The PCA algorithm effectively looks for the rotation that makes the dataset look "fattest" (maximises the variance), chooses that as the first dimension, then removes that dimension, rotates again to make it look "fattest" and repeats. Linear algebra makes it efficient to do this process in a single step by extracting the *eigenvectors* of the *covariance matrix*. 

PCA always finds a matrix $A$ such that $y = Ax$, where the dimension of $y<x$. PCA is exact and repeatable and very efficient, but it can only find rigid transformations of the data. This is a limitation of any linear dimensional reduction technique.




In [ ]:
digits = sklearn.datasets.load_digits()
digit_data = digits.data

# plot a single digit data element
def show_digit(d):
    plt.imshow(d.reshape(8,8), cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.figure()
    
# show a couple of raw digits
show_digit(digit_data[0])
show_digit(digit_data[400])

# apply principal component analysis
pca = sklearn.decomposition.PCA(n_components=2).fit(digit_data)
digits_2d = pca.transform(digit_data)

# plot each digit with a different color
plt.scatter(digits_2d[:,0], digits_2d[:,1], c=digits.target, cmap='jet')


One useful property of PCA is that we compute exactly how "fat" each of these learned dimensions were. The ratio of *explained variance* tells us how much each of the original variation in the dataset is captured by each learned dimension. 

If most of the variance is in the first couple of components, we know that a 2D representation will capture much of the original dataset. If the ratios of variance are spread out over many dimensions, we will need many dimensions to represent the data well. 

In [ ]:
# We can see how many dimensions we need to represent the data well using the eigenspectrum
# here we show the first 32 components
pca = sklearn.decomposition.PCA(n_components=32).fit(digit_data)
plt.bar(np.arange(32), pca.explained_variance_ratio_)
plt.xlabel("Component")
plt.ylabel("Proportion of variance explained")

One example where PCA does badly is the "swiss roll dataset" -- a plane rolled up into a spiral in 3D. This has a very simple structure; a simple plane with some distortion. But PCA can never unravel the spiral to find this simple explanation because it cannot be unravelled via a linear transformation.

In [ ]:
swiss_pos, swiss_val = sklearn.datasets.make_swiss_roll(800, noise=0.0)
fig = plt.figure()
# make a 3D figure
ax = fig.add_subplot(111, projection="3d")
ax.scatter(swiss_pos[:,0], swiss_pos[:,1], swiss_pos[:,2], c=swiss_val, cmap='gist_heat', s=10)


In [ ]:
# Apply PCA to learn this structure (which doesn't help much)
pca = sklearn.decomposition.PCA(2).fit(swiss_pos)
pca_pos = pca.transform(swiss_pos)
plt.scatter(pca_pos[:,0], pca_pos[:,1], c=swiss_val, cmap='gist_heat')

##### Manifold learning
Other approaches to dimensional reduction look at the problem in terms of learning a *manifold*. A *manifold* is a geometrical structure which is locally like a low-dimensional Euclidean space. Examples are the plane rolled up in the swiss roll, or a 1D "string" tangled up in a 3D space. 

Manifold approaches attempt to automatically find these smooth embedded structures by examining the local structure of datapoints (often by analysing the nearest neighbour graph of points). This is more flexible than linear dimensional reduction as it can in theory unravel very complex or tangled datasets. 

However, the algorithms are usually approximate, they do not give guarantees that they will find a given manifold, and can be computationally intensive to run.

A popular manifold learning algorithm is *ISOMAP* which uses nearest neighbour graphs to identify locally connected parts of a dataset.



In [ ]:
isomap_pos = sklearn.manifold.Isomap(n_neighbors=10, n_components=2).fit_transform(swiss_pos)
plt.scatter(isomap_pos[:,0], isomap_pos[:,1], c=swiss_val, cmap='gist_heat')
plt.figure()

# note that isomap is sensitive to noise!
noisy_swiss_pos, swiss_val = sklearn.datasets.make_swiss_roll(800, noise=0.5)
isomap_pos = sklearn.manifold.Isomap(n_neighbors=10, n_components=2).fit_transform(noisy_swiss_pos)
plt.scatter(isomap_pos[:,0], isomap_pos[:,1], c=swiss_val, cmap='gist_heat')


We can apply a more modern dimensional reduction method -- t-distributed stochastic neighbour embedding -- to the digits dataset we used in the PCA example.

In [ ]:
# This is very slow to run
tsne_digits = sklearn.manifold.TSNE(n_components=2).fit_transform(digit_data)
plt.scatter(tsne_digits[:,0], tsne_digits[:,1], c=digits.target, cmap='jet')

t-SNE separates out the digits very nicely into separate regions one for each digit, from the image data alone.

#### Clustering
Clustering tries to find well-seperated (in some sense) partitions of a data set. It is essentially a search for natural boundaries in the data. An example use of clustering might be gathering data from a large number of plants (e.g. measuring flower sizes, colouring, leaf symmetries, etc.) and then applying clustering to the data to try and separate out potential species groupings. 

There are many clustering approaches. A simple one is *k-means*, which finds clusters via an iterative algortihm. The number of clusters must be chosen in advance. In general, it is hard to estimate the number of clusters, although there are algorithms for estimating this. k-means proceeds by choosing a set of $k$ random points as initial cluster seed points; classifiying each data point according to its nearest seed point; then moving the cluster point towards the mean position of all the data points that belong to it. 

The k-means algorithm does not guarantee to find the best possible clustering -- it falls into *local minima*.



In [ ]:
# this is the classic "Iris" dataset, that captures 4 attributes measured from various irises (of three distinct species)
iris = sklearn.datasets.load_iris()

# we plot the dataset on two of the dimensions
plt.scatter(iris.data[:,2], iris.data[:,1], c=iris.target, cmap='jet')

plt.figure()
# now we cluster *without* any knowledge of the true class labels
kmeans = sklearn.cluster.KMeans(n_clusters=3)
kmeans_target = kmeans.fit_predict(iris.data)
plt.scatter(iris.data[:,2], iris.data[:,1], c=kmeans_target, cmap='jet')
# plot the cluster centres
plt.plot(kmeans.cluster_centers_[:,2], kmeans.cluster_centers_[:,1], 'r*', markersize=20)

## if we get the number of clusters wrong, the algorithm hallucinates clusters where there are none:
plt.figure()
# now we try and find 5 clusters
kmeans = sklearn.cluster.KMeans(n_clusters=5)
kmeans_target = kmeans.fit_predict(iris.data)
plt.scatter(iris.data[:,2], iris.data[:,1], c=kmeans_target, cmap='jet')
# plot the cluster centres
plt.plot(kmeans.cluster_centers_[:,2], kmeans.cluster_centers_[:,1], 'r*', markersize=20)



#### Agglomerative clustering
Another approach is to consider the clustering from bottom-up instead of top-down. *Agglomerative clustering* merges pairs of data points together, creating a tree of neighbour clusters. Once the preset number of leaves have been formed, these are returned as the clusters

In [ ]:
# we plot the dataset on two of the dimensions
plt.scatter(iris.data[:,2], iris.data[:,1], c=iris.target, cmap='jet')

plt.figure()
agg = sklearn.cluster.AgglomerativeClustering(n_clusters=3)
agg_target = agg.fit_predict(iris.data)
plt.scatter(iris.data[:,2], iris.data[:,1], c=agg_target, cmap='jet')


## Features, labels, data formats

### Types of data
#### Continuous
#### Ordinal
#### Categorical


### Feature vectors
#### Vector length
#### Selecting or generating features
#### Dealing with time series, text, images, sounds
### Labels
#### Multi-class approaches
###### One-vs-alll
###### One-vs-one

#### One-hot encoding
#### Hyperparameters
##### Learning rate
##### Smoothness
##### Momentum
##### Grid search
##### Using grid seaches effectively

### Learning
#### Batches, online learning, mini-batches
#### Loss functions
#### Gradient descent 
##### Stochastic gradient descent
#### Unbalanced data
#### Solutions for unbalanced data



## Evaluating machine learning



### Classifiers
#### Accuracy
#### Why is accuracy not enough?
#### FAR, FRR, F1-score
#### Receiver-operator curves
#### EER, AUC
#### Confusion matrices

### Unsupervised learning
#### Clustering metrics
#### 


In [ ]:
%cd datasets/uci
%ls

In [ ]:
# load the "Sonar" data set [https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)]
# This is a set of 60 sonar measurements, from bouncing sonar waves off either rocks ("R") or mines ("M")
# The problem is to tell the mines apart from the rocks
sonar_data = pd.read_csv("sonar.all-data")

# separate features
sonar_features = np.array(sonar_data)[:,0:60].astype(np.float64)

# we use label_binarize to convert "M" and "R" labels into {0,1}
# the ravel() just flattens the resulting 1D matrix into a vector
sonar_labels = sklearn.preprocessing.label_binarize(np.array(sonar_data)[:,60], classes=['M', 'R']).ravel()

# split into a train and test section, holding out 20% (0.2) of the data for testing
sonar_train_features, sonar_test_features, sonar_train_labels, sonar_test_labels = sklearn.cross_validation.train_test_split(
    sonar_features, sonar_labels, test_size=0.3, random_state=0)

# fit an SVM with the default parameters
svm = sklearn.svm.SVC(C=1)
svm.fit(sonar_train_features, sonar_train_labels)
print "Score: %f" % svm.score(sonar_test_features, sonar_test_labels)



In [ ]:
# we can print the confusion matrix
sonar_predicted_labels = svm.predict(sonar_test_features)
confusion_matrix =  sklearn.metrics.confusion_matrix(sonar_test_labels, sonar_predicted_labels)

# ipy_table.make_table just pretty prints the resulting matrix
ipy_table.make_table(confusion_matrix)

In [ ]:
# we can plot the receiver-operator curve: the graph of false positive rate against true positive rate
scores = svm.decision_function(sonar_test_features)
fpr, tpr, thresholds = sklearn.metrics.roc_curve(sonar_test_labels, scores)
plt.plot(fpr,tpr)
plt.plot([0,1], [0,1])
plt.plot([0,1], [1,0])
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.legend(["ROC", "Chance"])

In [ ]:
print "AUC=%f" % sklearn.metrics.roc_auc_score(sonar_test_labels, scores)
print "Accuracy=%f" % sklearn.metrics.accuracy_score(sonar_test_labels, sonar_predicted_labels)
# recall -- how many of the positive samples did it find?
print "Recall=%f" % sklearn.metrics.recall_score(sonar_test_labels, sonar_predicted_labels)
# precision  -- how many of the negative samples did it reject?
print "Precision=%f" % sklearn.metrics.precision_score(sonar_test_labels, sonar_predicted_labels)

# F1 =  2 * (precision * recall) / (precision + recall)
print "F1 score=%f" % sklearn.metrics.f1_score(sonar_test_labels, sonar_predicted_labels)

In [ ]:
print sklearn.metrics.classification_report(sonar_test_labels, sonar_predicted_labels)

In [ ]:

# let's search for the best parameters of C and gamma
# we create a grid of parameter values, log-spaced across a wide range
C_vals = np.logspace(-2,8,10)
gamma_vals = np.logspace(-4,3,10)

# 10x10 values of C and gamma; 10 fold cross validation for each
grid_cv = sklearn.grid_search.GridSearchCV(svm, {'C':C_vals, 'gamma':gamma_vals}, cv=10)
grid_cv.fit(sonar_train_features, sonar_train_labels)


In [ ]:
# print the optimal best results
print grid_cv.best_params_
print grid_cv.best_score_


In [ ]:
# fit an SVM using the optimal parameters found
svm = sklearn.svm.SVC(C=grid_cv.best_params_['C'], gamma=grid_cv.best_params_['gamma'])
svm.fit(sonar_train_features, sonar_train_labels)
print "Score: %f" % svm.score(sonar_test_features, sonar_test_labels)

In [ ]:
# We can randomly permute the label vectors using np.random.permutation
# Now they have no relation to the training features
random_sonar_train_labels = np.random.permutation(sonar_train_labels)
random_sonar_test_labels = np.random.permutation(sonar_test_labels)


In [ ]:
# fit an SVM to the randomised label data
svm = sklearn.svm.SVC(C=15, gamma=0.12)
svm.fit(sonar_train_features, random_sonar_train_labels)
# this should be about 0.5
print "Score: %f" % svm.score(sonar_test_features, random_sonar_test_labels)

## Overfitting
Overfitting is the key issue with machine learning algorithms. It is trivially easy to devise a supervised learning algorithm that takes in input features and exactly predicts the corresponding output classes *in the training data*. A simple lookup table will do this.

To make useful predictions, a learning algorithm must predict values with features it has never seen. The problem is we can only optimise the performance based on data we *have* seen. The **generalisation** performance of an algorithm is the ablility to make predictions outside of the training data. 

This means that we cannot optimise an algorithm by adjusting parameters to fit the training data better; this will lead to **overfitting**, where the predictive power of the algorithm *decreases* as it is exposed to additional data. 

Instead, we must use a separate testing set which the training algorithm is never exposed to to evaluate performance. The importance of this can be shown in a few simple examples.

#### Polynomial overfitting
A simple example of overfitting can be seen when fitting a polynomial $y=a_0x^0 + a_1x^1 + \dots + a_qx^q$ to $n$ points. When $n=q$, the curve goes through all the points, but makes wildly inappropriate interpolations between them.

The example below shows this effect:

In [ ]:
# create some random points
n_points = 6
x = np.random.uniform(-1,1,(n_points,))
y = np.random.uniform(-1,1,(n_points,))

In [ ]:
# plot the points
plt.scatter(x,y);

We define a short function to fit and plot a polynomial fit . NumPy has a built in function `polyfit` which does least-squares fitting).

In [ ]:
def plot_poly_fit(x,y,order):    
    plt.figure()
    plt.scatter(x,y)
    # fit a polynomial
    coeffs = np.polyfit(x,y,order)
    # evaluate it across the domain
    xs = np.linspace(np.min(x),np.max(x),100)
    ys = np.polyval(coeffs, xs)
    plt.plot(xs,ys)
    

In [ ]:
plot_poly_fit(x,y,order=1) # order=1 means a linear fit

Try running the above code with increasing values for the order

#### Kernel density estimation example
We can see the same effect if we try to learn the distribution of data using **kernel density estimation** (KDE). KDE is effectively a smoothed histogram, which is created by "placing" smooth distributions on each observed data point and summing them (i.e. convolving them with some window function). This can be used to estimate an underlying smooth distribution from point samples.

The key parameter in KDE is the **kernel width** $\sigma$, which determines how wide each distribution will be and thus how smooth the overall distribution will be.

In [ ]:
# generate some random numbers -- the use of the  beta distribution isn't important, it just gives an interesting shape
x = np.random.beta(0.5, 2, 40);
# plot the data points
plt.figure()
# scatter plot showing the actual positions
plt.scatter(x,np.ones_like(x))
plt.figure()
# histogram of the data points
plt.hist(x, normed=True);

In [ ]:
import scipy.stats as stats 

# plot the kernel density estimate (Gaussian window) with the given bandwidth
def plot_kde(x, width):
    kde = stats.kde.gaussian_kde(x, bw_method=width)
    # evaluate kde estimate over range of x
    xs = np.linspace(np.min(x), np.max(x), 100) 
    plt.figure()
    plt.plot(xs, kde(xs))
    plt.scatter(x, np.ones_like(x))


In [ ]:
plot_kde(x, 2) # too smooth
plot_kde(x, 1) # a bit too smooth
plot_kde(x, 0.5) # good
plot_kde(x, 0.1) # too rough
plot_kde(x, 0.01) # just learning the data points



As the function approximates the data better, the generalisation performance drops. If we split the data randomly into two portions $X_1$ and $X_2$, and learn the KDE using only $X_1$ (training set) and then compute how likely $X_2$ (test set) is given that learned distribution, we can see this loss of generalisation performance.


In [ ]:
def split_data(x):
    # our data here is random and uncorrelated, so we can just split the array into two
    l = len(x)//2
    return x[:l], x[l:]

def learn_kde(x, width):
    return stats.kde.gaussian_kde(x,width)

def evaluate_kde(x, kde):
    # we can compute the log-likelihood by summing the log pdf evaluated at x
    return np.sum(kde.logpdf(x))


In [ ]:
def test_kde(x, width):
    # split the data into two parts, train on one, and then test it on both of the splits
    x1,x2 = split_data(x)
    kde = learn_kde(x1, width)
    # return the train log-likelihood and test log-likelihood    
    return evaluate_kde(x1, kde), evaluate_kde(x2, kde)


In [ ]:
def plot_kde_lik(x):
    # plot test and train log-likelihood as a function of 1/sigma
    widths = np.linspace(0.05, 20, 100)
    trains = []
    tests = []
    # test a bunch of widths
    for width in widths:
        train, test = test_kde(x,1.0/width)
        trains.append(train)
        tests.append(test)
        
    # plot and label
    plt.plot(widths, trains)
    plt.plot(widths, tests)
    plt.xlabel("$\sigma^{-1}$")
    plt.ylabel("Log-likelihood")
    plt.legend(["Training", "Test"])
    

We can plot the log-likelihood as a function of $\sigma^{-1}$ (the reciprocal simply makes it easier to see what is going on in the graph).

In [ ]:
plot_kde_lik(x)

### Training error can always be reduced -- but it makes things worse
We can see that test and training become better as $\sigma^{-1}$ approaches 2-4 (the exact value will depend on what random numbers we originally drew), then rapidly decreases; but the training log-likelihood **always** increases as we approximate the original data better and better.

## Data hygiene
This is why **data hygiene** is absolutely critical. If you let any part of the data you use to evaluate performance affect the train process your results are *meaningless*. 

#### Randomised selection
One approach to splitting up data is to randomly assign some elements to the training set and some to the test set (e.g. in an image classification task, 70% of the images are assigned to the training class and 30% to the test class). 

This seems like an unbiased way of separating the data, and it is for problems which are effectively uncorrelated. But imagine we have a time series $x_0, x_1, \dots, x_n$ and we build our input features $X_0, X_1, \dots$ by taking overlapping windows of the series. If we randomly choose elements of $X$, many of the elements in the test and training set may be almost identical ( because they appeared next to each other in the time series). This leads to wildly optimistic test results


#### Block selection
A much better approach here is to split the data into a large chunks. Say the data was a series of field recordings of birds taken on 10 different days; the first 6 days might be assigned as training and the last 4 as test. 

This is much more likely to be a reliable estimator of future performance, because the key idea is to *predict future behaviour* -- to learn what we have not seen. Choosing the split of training and test requires thought and domain knowledge and it is critical to make sure that the evaluation results are meaningful.



### Cross validation
Sometimes the data is too precious to split into training and test sets; there simply isn't enough of it. 

One approach to getting reliable results without overfitting is to use *cross validation*. This simply involves splitting the data into a test and training set repeatedly and averaging the results. 

#### k-fold cross validation
*k-fold* cross validation splits the data into $k$ blocks (again the block selection strategy needs to be chosen carefully to avoid overfitting) and then trains on $k-1$ of the blocks and test on the remaining one, for each of the $k$ test blocks.
#### Leave-one-out cross validation (LOOCV)
LOOCV takes this to an extreme, and splits the data into $k=n$ blocks (for $n$ data points) and trains on all but one data point and tests on the one that was left out. This is a reliable estimator of performance but can be very computationally expensive, and the benefits over $k$-fold cross validation are not always great.

 



In [ ]:
# k-fold cross validation with the KDE example

def k_split(x, k):
    # split the data into even chunks
    l = len(x)//k
    return [x[i*l:(i+1)*l] for i in range(k)]

    

In [ ]:
def test_kde_k_fold(x, width, k):
    # split the data into two parts, train on one, and then test it on both of the splits
    liks = []
    folds = k_split(x,k)
    for kn in range(k):
        # concatenate all folds *but* kn
        x1 = np.concatenate([folds[i] for i in range(k) if i!=kn])            
        # test set is the one we left out
        x2 = folds[kn]
        # fit the KDE
        kde = learn_kde(x1, width)
        # return the log-likelihood for this fold
        lik = evaluate_kde(x2, kde)
        liks.append(lik)
    return liks

In [ ]:
# now we can plot the performance with one std. dev. bounds

def plot_kde_lik_k_fold(x, k):
    # plot test and train log-likelihood as a function of 1/sigma
    widths = np.linspace(0.05, 20, 100)
    means = []
    stds = []
    # test a bunch of widths
    for width in widths:
        ks = test_kde_k_fold(x,1.0/width,k)
        means.append(np.mean(ks))
        stds.append(np.std(ks))
                
    # plot and label    
    means = np.array(means) # convert lists no numpy arrays so we can do arithmetic on them
    stds = np.array(stds)
    
    plt.fill_between(widths, means-stds, means+stds, alpha=0.1)
    plt.plot(widths, means)
    plt.xlabel("$\sigma^{-1}$")
    plt.ylabel("Log-likelihood")
    

In [ ]:
plot_kde_lik_k_fold(x, 5)

### Validation sets
Often your learning algorithms might have hyperparameters you need to estimate (e.g. learning rate). 

You absolutely **cannot** use the test set to evaluate the performance with various parameter settings and then choose the best one. This will overfit, because the test set is influencing the training performance. Train and test must be **completely** separated; no information may flow from the test data to the training process

Instead, you can create a **validation** set from the training set, and use that to tweak the hyperparameters. One common and sensible approach is to first separate off a single, fixed test set, and then use cross-validation to create multiple train and validation sets. 

#### Nested cross-validation
*Nested cross-validation* applies cross validation to both the validation and test sets, first fixing a fold split for the test set, then optimising the hyperparameters using cross-validation on the remaining data, then moving onto the next fold split for the test set, and so on.



### Baselines
Let's return to our classification example. What is good performance? Or more particularly, what is bad performance -- what's the baseline? A very simple test of whether the classifier is learning **anything** useful is to simply randomly permute the targets so they no longer correspond to the inputs. 

This estimates the chance performance of the classification process. It might be that you have a binary classification problem with exactly balanced datasets, so the baseline is 50%; but often the data is unbalanced and there are multiple classes. Permuting the targets is a very quick way to test the random performance.


#### An unbalanced example
As a simple example, we can look at a very unbalanced classifcation problem. The trees dataset includes measurements from aerial photography of land cover. A small portion of it is diseased tree growth. We can train a classifier to predict the disease status from the measurements.

In [ ]:
# load the "wilt" data set https://archive.ics.uci.edu/ml/datasets/Wilt
wilt_data = pd.read_csv("wilt.csv")
# separate features
wilt_features = wilt_data.ix[:,1:].as_matrix()

# we use label_binarize to convert "M" and "R" labels into {0,1}
# the ravel() just flattens the resulting 1D matrix into a vector
wilt_labels = sklearn.preprocessing.label_binarize(np.array(wilt_data.ix[:,0]), classes=['w', 'n']).ravel()

# split into a train and test section, holding out 20% (0.2) of the data for testing
wilt_train_features, wilt_test_features, wilt_train_labels, wilt_test_labels = sklearn.cross_validation.train_test_split(
    wilt_features, wilt_labels, test_size=0.3, random_state=0)

# fit an SVM with the default parameters
wilt_svm = sklearn.svm.SVC(C=10000, gamma=5.0)
wilt_svm.fit(wilt_train_features, wilt_train_labels)
print "Score: %f" % wilt_svm.score(wilt_test_features, wilt_test_labels)



Great! 95% accuracy!
Let's try it with random labels:

In [ ]:
wilt_svm = sklearn.svm.SVC(C=1)
wilt_svm.fit(wilt_train_features, np.random.permutation(wilt_train_labels))
print "Score: %f" % wilt_svm.score(wilt_test_features, np.random.permutation(wilt_test_labels))


The SVM learned essentially **nothing**; the accuracy is very misleading. 

In [ ]:
scores = wilt_svm.decision_function(wilt_test_features)
fpr, tpr, thresholds = sklearn.metrics.roc_curve(wilt_test_labels, scores)
plt.plot(fpr,tpr)
plt.plot([0,1], [0,1])
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.legend(["ROC", "Chance"])

## Meta-algorithms
Meta-algortihms are ways of combining multiple learning models to improve performance. 
These generally involve *ensembles* of classifiers. They can be applied to a very wide range of strategies, and they *generally always improve performance* (even if the improvement is marginal). In major machine learning competitions (e.g. Kaggle, the Netflix challenge) *ensemble* algorithms are almost always the top performers.

The idea of a *ensemble* method is that if you train multiple classifiers/regressors of different types or on different datasets, they will learn different things well; and combining them together increases the robustness and generalisation performance. 

### Voting hybrid models
One simple model is to train a number of different types of classifiers on a dataset and have them vote on the class label. For regression, the median or mean can be used as the combination function.

#### Weighted models
### Bagging
#### Boosting

In [ ]:
# voting model on the sonar dataset

# Fit SVM
svm = sklearn.svm.SVC(C=12, gamma=0.3)
svm.fit(sonar_train_features, sonar_train_labels)
print "SVM Score: %f" % svm.score(sonar_test_features, sonar_test_labels)

# Fit KNN
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=3)
knn.fit(sonar_train_features, sonar_train_labels)
print "KNN Score: %f" % knn.score(sonar_test_features, sonar_test_labels)

# Fit decision tree
dec = sklearn.tree.DecisionTreeClassifier()
dec.fit(sonar_train_features, sonar_train_labels)
print "DEC Score: %f" %dec.score(sonar_test_features, sonar_test_labels)

# predict labeels
svm_labels = svm.predict(sonar_test_features)
knn_labels = knn.predict(sonar_test_features)
dec_labels = dec.predict(sonar_test_features)

# to vote, we can take the mean and use 0.5 as a threshold
mean_labels = np.mean(np.vstack((svm_labels, knn_labels, dec_labels)), axis=0)
voted = np.where(mean_labels>0.5, 1, 0)

print "Voted score: %f" % sklearn.metrics.accuracy_score(sonar_test_labels, voted)



We can also *weight* classifiers by how well they are doing; strong classifiers should be weighted more.

### Bagging
Rather than combining different models, we can use a single model but different *datasets*. Bagging applies the statistical process called the *bootstrap* to generate multiple classifiers/regressors. 

Bootstrap generates *synthetic datasets* by randomly resampling the original dataset **with replacement**. From a dataset $X$ with $n$ elements, it generates $k$ new datasets each of which have $n$ elements consisting of random draws from the rows of $X$. Bagging just trains one classifier for each of these $k$ datasets then combines the output by voting or averaging. This increases the robustness of the model.

This has the advantage of working for *any* supervised learning task (but there may be significant computational issues if the datasets are very large). However, bagging may not improve (or may even make worse) classifiers that are not already overfitting. 

Variations of this approach include randomly sampling the features (columns of $X$); random feature selection is called *random subspaces* and randomly sampling both features and samples is known as *random patches*.


In [ ]:
# create some random points
n_points = 10
x = np.random.uniform(-1,1,(n_points,))
y = np.random.uniform(-1,1,(n_points,))

# we return to the polynomial fitting example
def bootstrap_sample(x,y):
    # resample with replacement
    indices = np.random.randint(0, len(y), len(y))
    return x[indices], y[indices]

def plot_poly_fit_bagged(x,y,order,n_bags):    
    plt.figure()
    plt.scatter(x,y)
    
    all_ys = []
    
    # fit polynomials to resampled datasets
    for i in range(n_bags):
        bx, by = bootstrap_sample(x,y)
        # fit a polynomial
        coeffs = np.polyfit(bx,by,order)
        
        # evaluate it across the domain
        xs = np.linspace(np.min(x),np.max(x),100)
        ys = np.polyval(coeffs, xs)
        all_ys.append(ys)
    
    # convert to array
    all_ys = np.array(all_ys)
    
    for ys in all_ys:
        plt.plot(xs,ys, alpha=0.05)
    plt.plot(xs, np.median(all_ys, axis=0))
    plt.ylim(-2,2)
    
plot_poly_fit(x,y, 4)
plot_poly_fit_bagged(x,y,4,100)
    


In [ ]:
# We can try bagging the mine data set

# Fit poorly generalising KNN
knn = sklearn.neighbors.KNeighborsClassifier(2)
bagged_knn = sklearn.ensemble.BaggingClassifier(knn, n_estimators=50)

knn.fit(sonar_train_features, sonar_train_labels)
print "KNN Score: %f" % knn.score(sonar_test_features, sonar_test_labels)

bagged_knn.fit(sonar_train_features, sonar_train_labels)
print "Bagged KNN Score: %f" % bagged_knn.score(sonar_test_features, sonar_test_labels)

#### Random forests
*Random forests* use ensembles of very simple classifiers -- decision trees -- to transform a weak and poorly generalising learning model into a much more robust model. Random forests often have very competitive performance in classification tasks.


In [ ]:
# Run the test 10 times
for trials in range(20):
    forest_scores = []
    # iterate over number of trees in the ensemble
    for estimators in range(20):
        # completely randomised tree ensemble classifiers
        forest = sklearn.ensemble.ExtraTreesClassifier(n_estimators=estimators+1)
        forest.fit(sonar_train_features, sonar_train_labels)
        forest_scores.append(forest.score(sonar_test_features, sonar_test_labels))
    plt.plot(np.arange(len(forest_scores))+1, forest_scores, alpha=0.2)
plt.xlabel('No. estimators')
plt.ylabel('Accuracy')

#### Boosting
*Boosting* is an alternative ensemble method which trains a weak classifier on a dataset, identifies samples it is performing poorly on, and trains another classifier to learn the poor samples, identifies samples the ensemble is still performing poorly on, and trains a classifer to learn **those** and so on. 

The selection of the weak samples is usually done by *weighting* the samples rather than a binary inclusion/exclusion. The AdaBoost algorithm is a well-known example of this class, and can combine weak learners such as decision stumps (a decision tree with only one decision!) into effective classifiers. 

In [ ]:
dec = sklearn.tree.DecisionTreeClassifier(max_depth=1)
dec.fit(sonar_train_features, sonar_train_labels)
boost_scores = []
for estimators in range(50):
    boosted_dec = sklearn.ensemble.AdaBoostClassifier(dec, n_estimators=estimators+1, algorithm='SAMME')
    boosted_dec.fit(sonar_train_features, sonar_train_labels)
    boost_scores.append(boosted_dec.score(sonar_test_features, sonar_test_labels))
plt.axhline(dec.score(sonar_test_features, sonar_test_labels))
plt.plot(np.arange(len(boost_scores))+1, boost_scores)
plt.xlabel('No. estimators')
plt.ylabel('Accuracy')